In [25]:
import pandas as pd
import random
import time
from numpy.random import choice
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from datetime import datetime
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable

In [26]:
#https://stackoverflow.com/questions/553303/how-to-generate-a-random-date-between-two-other-dates
def str_time_prop(start, end, time_format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return datetime.fromtimestamp(ptime)


def random_date(start, end, prop):
    return str_time_prop(start, end, '%d/%m/%Y %H:%M', prop)

In [27]:
def get_normal_indexes(sample_size, example_size, spread=6):
    mu = example_size/2
    sigma = example_size/spread
    return np.clip([int(i) for i in np.random.normal(mu, sigma, sample_size)], 0, example_size-1)

# Warning!
- #### If you run the below cells the data set will be recreated and replaced
- #### To run the analysis go to the analysis section

In [33]:
n_rows = int(1e5)
columns_names = [
    "Data da Venda",
    "Região",
    "Vendedor",
    "Produto",
    "Categoria do Produto",
    "Valor do Produto",
    "Quantidade Vendida",
    "Forma de Pagamento",
    "Status do Pedido",
    "Cliente_ID",
    ]

In [34]:
# regioes distribuition is based on population %
regioes = ["Sul", "Sudeste", "Centro-Oeste", "Nordeste", "Norte"]
regioes_dist = [0.15, 0.42, 0.08, 0.27, 0.08]

In [35]:
vendedores = ["Helena","Cecília","Maitê","Laura","Alice","Miguel","Artur","Davi","Antônio","Samuel"]

In [36]:
produtos = [
    #Eletrodomésticos
    {"Produto": "Geladeira Brastemp 375L", "Categoria do Produto": "Eletrodomésticos", "Valor": 3850.00},
    {"Produto": "Micro-ondas Electrolux", "Categoria do Produto": "Eletrodomésticos", "Valor": 620.00},
    {"Produto": "Fogão Consul 4 bocas", "Categoria do Produto": "Eletrodomésticos", "Valor": 1150.00},
    {"Produto": "Máquina de Lavar 11kg", "Categoria do Produto": "Eletrodomésticos", "Valor": 2300.00},
    
    # Eletrônicos
    {"Produto": "Câmera Canon EOS Rebel", "Categoria do Produto": "Eletrônicos", "Valor": 3200.00},
    {"Produto": "Smart TV Samsung 50\"", "Categoria do Produto": "Eletrônicos", "Valor": 2900.00},
    {"Produto": "Console PlayStation 5", "Categoria do Produto": "Eletrônicos", "Valor": 4499.00},
    {"Produto": "Drone DJI Mini 2", "Categoria do Produto": "Eletrônicos", "Valor": 3499.00},
    
    # Esportes
    {"Produto": "Esteira Elétrica Movement", "Categoria do Produto": "Esportes", "Valor": 5990.00},
    {"Produto": "Patins Rollerblade Zetrablade", "Categoria do Produto": "Esportes", "Valor": 899.00},
    {"Produto": "Bola de Futebol Nike Campo", "Categoria do Produto": "Esportes", "Valor": 230.00},
    {"Produto": "Bicicleta Caloi 100", "Categoria do Produto": "Esportes", "Valor": 2100.00},

    # Telefonia
    {"Produto": "Smartphone iPhone 14", "Categoria do Produto": "Telefonia", "Valor": 7499.00},
    {"Produto": "Smartphone Motorola Edge 40", "Categoria do Produto": "Telefonia", "Valor": 3599.00},
    {"Produto": "Fone Bluetooth Samsung Buds 2", "Categoria do Produto": "Telefonia", "Valor": 699.00},
    {"Produto": "Smartphone Samsung A34", "Categoria do Produto": "Telefonia", "Valor": 1799.00},

    # Informática
    {"Produto": "SSD Kingston NV2 1TB", "Categoria do Produto": "Informática", "Valor": 480.00},
    {"Produto": "Placa de Vídeo Nvidia RTX 4060", "Categoria do Produto": "Informática", "Valor": 2399.00},
    {"Produto": "Notebook Dell Inspiron", "Categoria do Produto": "Informática", "Valor": 4500.00},
    {"Produto": "Monitor LG 24\"", "Categoria do Produto": "Informática", "Valor": 1200.00},

    # Áudio
    {"Produto": "Soundbar Samsung HW-T450", "Categoria do Produto": "Áudio", "Valor": 999.00},
    {"Produto": "Headset Gamer HyperX Cloud II", "Categoria do Produto": "Áudio", "Valor": 520.00},
    {"Produto": "Fone de Ouvido JBL", "Categoria do Produto": "Áudio", "Valor": 350.00},
    {"Produto": "Caixa de Som JBL Go 3", "Categoria do Produto": "Áudio", "Valor": 230.00},

    # Eletroportáteis
    {"Produto": "Cafeteira Nespresso Essenza Mini", "Categoria do Produto": "Eletroportáteis", "Valor": 520.00},
    {"Produto": "Air Fryer Mondial 4L", "Categoria do Produto": "Eletroportáteis", "Valor": 430.00},
    {"Produto": "Ventilador Mondial 40cm", "Categoria do Produto": "Eletroportáteis", "Valor": 200.00},
    {"Produto": "Aspirador de Pó Philco", "Categoria do Produto": "Eletroportáteis", "Valor": 420.00},

    # Acessórios
    {"Produto": "Carregador Portátil Anker 10000mAh", "Categoria do Produto": "Acessórios", "Valor": 180.00},
    {"Produto": "Óculos de Sol Ray-Ban Wayfarer", "Categoria do Produto": "Acessórios", "Valor": 620.00},
    {"Produto": "Relógio Smartwatch Xiaomi", "Categoria do Produto": "Acessórios", "Valor": 480.00},
    {"Produto": "Mochila Lenovo para notebook", "Categoria do Produto": "Acessórios", "Valor": 150.00},

    # Impressoras
    {"Produto": "Impressora Epson EcoTank L3250", "Categoria do Produto": "Impressoras", "Valor": 1249.00},
    {"Produto": "Impressora HP DeskJet", "Categoria do Produto": "Impressoras", "Valor": 480.00},

    # Vestuário
    {"Produto": "Jaqueta Corta Vento Adidas", "Categoria do Produto": "Vestuário", "Valor": 450.00},
    {"Produto": "Calça Legging Nike Pro", "Categoria do Produto": "Vestuário", "Valor": 280.00},
    {"Produto": "Boné New Era Yankees", "Categoria do Produto": "Vestuário", "Valor": 180.00},
    {"Produto": "Tênis Nike Revolution", "Categoria do Produto": "Vestuário", "Valor": 320.00},

    # Periféricos
    {"Produto": "Mouse Gamer Razer DeathAdder V2", "Categoria do Produto": "Periféricos", "Valor": 320.00},
    {"Produto": "Teclado Logitech K380 Bluetooth", "Categoria do Produto": "Periféricos", "Valor": 250.00},
    {"Produto": "Mouse Logitech M170", "Categoria do Produto": "Periféricos", "Valor": 75.00},
    {"Produto": "Teclado Mecânico Redragon", "Categoria do Produto": "Periféricos", "Valor": 250.00},
]
max_value = max([v['Valor'] for v in produtos])
produtos_dist = np.array([100+max_value-p['Valor'] for p in produtos])
normalized_produtos_dist = produtos_dist / produtos_dist.sum()

#Giving a normal distribuition on products
products_indexes = get_normal_indexes(n_rows, len(produtos), spread=3)

In [37]:
formas_de_pagamento = ["Crédito", "Débito", "Boleto", "Pix", None]
formas_de_pagamento_dist = [0.5, 0.1, 0.1, 0.25, 0.05]

In [38]:
#Givin more probability for "Pago"
status = ["Pago", "Aguardando Pagamento", "Em andamento", "Em Atraso", "Cancelado"]
status_dist =  [0.7, 0.05, 0.1, 0.1, 0.05]

In [40]:
n_clients = 1000
clientes = [f"{random.choice(range(100)):04n}" for i in range(n_clients)]

#Giving a normal distribuition on clients
clients_indexes = get_normal_indexes(n_rows, len(clientes))

In [41]:
# Datas inicial e final
start = datetime.strptime('1/1/2020 00:00', '%d/%m/%Y %H:%M')
end = datetime.strptime('31/12/2024 00:00', '%d/%m/%Y %H:%M')

# Gerar lista de n datetimes igualmente espaçados
datas = pd.date_range(start=start, end=end, periods=(end-start).days).to_list()

# The most recent dates have more probability of get choose, so we see an increase on sales
sale_increase = 2
data_dist = np.array([(x / (end-start).days)**2 for x in range((end-start).days, sale_increase*(end-start).days, sale_increase-1)])
normalized_data_dist = data_dist / data_dist.sum()

In [42]:
# Creating the data columns
data_venda_list = choice(a=datas, size=n_rows, p=normalized_data_dist)
data_venda_list = [t.strftime('%d/%m/%Y %H:%M') for t in data_venda_list]

regiao_list = choice(a=regioes, size=n_rows, p=regioes_dist)

vendedores_list = [random.choice(vendedores) for i in range(n_rows)]

produtos_list = [p.values() for p in choice(a=produtos,size=n_rows, p=normalized_produtos_dist)]

quantidades_list = [random.choice(range(10)) for i in range(n_rows)]

formas_de_pagamento_list = choice(a=formas_de_pagamento, size=n_rows, p=formas_de_pagamento_dist)

status_do_pedido_list = choice(a=status,size=n_rows, p=status_dist)

clientes_list = [clientes[i] for i in clients_indexes]

In [43]:
data = [data_venda_list, regiao_list, vendedores_list]+list(map(list, zip(*produtos_list)))+[quantidades_list, formas_de_pagamento_list, status_do_pedido_list, clientes_list]
data = list(map(list, zip(*data)))

#### Inconsistencies
The raw data has the following inconsistencies added on purpose:
- The 'quantidades' value may be zero
- There are combinations of 'status' and 'formas de pagamento' that may be invalid:
    -  "Pago" with None
    -  "Cancelado" or "Aguardando Pagamento" with any other then None
- Added some None values on columns "Data da Venda", "Região", "Vendedor", "Produto" and "Status do Pedido"
- Added some random duplicates values

In [44]:
df = pd.DataFrame(
    data=data,
    columns=columns_names,
    index=None,
)

# Columns where you want to insert None
target_columns = ["Data da Venda", "Região", "Vendedor", "Produto", "Status do Pedido"]

# Adding 1% of None values per column
n_none_per_column = round(n_rows*0.01)

# Add None randomly
for col in target_columns:
    rows = random.sample(range(df.shape[0]), n_none_per_column)
    df.loc[rows, col] = None

# Adding 0.5% of duplicates rows
n_duplicates = round(n_rows*0.005)
# Randomly select rows to duplicate (with replacement if needed)
rows_to_duplicate = df.sample(n=n_duplicates, replace=True)

# Append duplicated rows
df = pd.concat([df, rows_to_duplicate], ignore_index=True)

In [45]:
df.to_excel("raw_data2.xlsx", index=None)